In [1]:
import pandas as pd
from utils import create_connection

In [2]:
conn = create_connection('data/airbnb.db',connection_type = 'sqlite3')

In [3]:
tables_df = pd.read_sql("""SELECT name FROM sqlite_master  WHERE type='table';""",con=conn)
tables_df

,name
0,airbnb_room
1,airbnb_calendar
2,airbnb_rooms
3,airbnb_room_details
4,airbnb_room_details_updates
5,airbnb_room_calendar_updates
6,airbnb_room_calendar_day_transitions
7,airbnb_room_calendar_days


In [15]:
airbnb_room_df = pd.read_sql("""select * 
                             from airbnb_room 
                             where True
                             --and id = '18086628'
                             
                             order by last_modified desc""",con=conn)

In [16]:
airbnb_room_df

,id,create_date,last_modified,number_updates,room_url
0,31335624,2024-06-12 07:32:50,2024-08-26 16:01:59,21,www.airbnb.com/rooms/31335624?adults=2&childre...
1,817196070864448954,2024-06-12 07:32:50,2024-08-26 16:01:59,19,www.airbnb.com/rooms/817196070864448954?adults...
2,24883185,2024-06-12 07:32:50,2024-08-26 16:01:59,31,www.airbnb.com/rooms/24883185?adults=2&childre...
3,34808908,2024-06-12 07:32:50,2024-08-26 16:01:59,13,www.airbnb.com/rooms/34808908?adults=2&childre...
4,1074724472918633842,2024-06-12 07:32:50,2024-08-26 16:01:59,29,www.airbnb.com/rooms/1074724472918633842?adult...
...,...,...,...,...,...
1958,1047051915405690952,2024-06-12 13:59:51,None,0,www.airbnb.com/rooms/1047051915405690952?adult...
1959,1177294408721270246,2024-06-12 14:51:57,None,0,www.airbnb.com/rooms/1177294408721270246?adult...
1960,765918426420984116,2024-06-12 14:51:58,None,0,www.airbnb.com/rooms/765918426420984116?adults...
1961,1004109262864251827,2024-06-12 14:51:58,None,0,www.airbnb.com/rooms/1004109262864251827?adult...


In [8]:
airbnb_room_df

,id,create_date,last_modified,number_updates,room_url
0,18086628,2024-06-12 07:32:51,2024-08-26 15:35:35,15,www.airbnb.com/rooms/18086628?adults=2&childre...


In [134]:
current_calendar_df= pd.read_csv('temp_df.csv')
current_calendar_df.drop_duplicates(subset='date', keep='first')
current_calendar_df['date'] = pd.to_datetime(current_calendar_df['date']).dt.date
current_calendar_active_days_df = current_calendar_df[~current_calendar_df['day_disabled']].copy()
current_calendar_active_days_df['days_group'] = ((current_calendar_active_days_df['date'] - current_calendar_active_days_df['date'].shift(1)) != pd.Timedelta(days=1)).cumsum()
current_calendar_active_days_groups = current_calendar_active_days_df.groupby(['days_group']).agg({'date':['min','max']}).reset_index()
current_calendar_active_days_groups.columns = [f"{i[0]}_{i[1]}" if i[1] else i[0] for i in current_calendar_active_days_groups.columns]

In [139]:
current_calendar_df[current_calendar_df['reason_string'].str.contains('Available, but has no eligible checkout date')].reason_string.unique()

array(['Available, but has no eligible checkout date, due to the 2 night stay requirement.'],
      dtype=object)

In [142]:
current_calendar_active_days_df.reason_string.unique()

array(['Available. There is a 1 night minimum stay requirement. Select as check-in date.',
       'Available, but has no eligible checkout date, due to the 2 night stay requirement.',
       'This day is only available for checkout.'], dtype=object)

In [133]:
current_calendar_active_days_groups.columns

Index(['days_group', 'date_min', 'date_max'], dtype='object')

In [111]:
temp_df

,day_disabled,date,reason_string
0,True,2024-08-01,Past dates can’t be selected.
1,True,2024-08-02,Past dates can’t be selected.
2,True,2024-08-03,Past dates can’t be selected.
3,True,2024-08-04,Past dates can’t be selected.
4,True,2024-08-05,Past dates can’t be selected.
...,...,...,...
238,True,2025-03-27,Unavailable
239,True,2025-03-28,Unavailable
240,True,2025-03-29,Unavailable
241,True,2025-03-30,Unavailable


In [63]:
a_df = pd.DataFrame(a)

In [67]:
%%timeit
a_df[~a_df['reason_string'].isin(['Past dates can’t be selected.'])]

163 μs ± 10.7 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [68]:
a_df[~a_df['reason_string'].isin(['Past dates can’t be selected.'])]

,day_disabled,date,reason_string
32,True,2024-09-02,Unavailable
33,True,2024-09-03,Unavailable
34,True,2024-09-04,Unavailable
35,True,2024-09-05,Unavailable
36,True,2024-09-06,Unavailable
...,...,...,...
238,True,2025-03-27,Unavailable
239,True,2025-03-28,Unavailable
240,True,2025-03-29,Unavailable
241,True,2025-03-30,Unavailable


In [9]:
airbnb_room_df = pd.read_sql("""select * 
                             from airbnb_room_calendar_days 
                             where True
                             """,con=conn)
airbnb_room_df[airbnb_room_df['state'] == 'AVAILABLE']

,room_id,calendar_day,created_at,updated_at,state,previous_state,minimum_stay_nights,price,previous_price,latest_prices_array,cleaning_fee,currency,extra_attributes
49,14132224,2024-10-20,2024-09-15 13:41:16,None,AVAILABLE,None,1,72.0,None,"[{""check_in"": ""2024-10-20"", ""check_out"": ""2024...",25.0,£,"{"""": ""Airbnb service fee\nShow price breakdown..."
62,14132224,2024-11-02,2024-09-15 13:41:16,None,AVAILABLE,None,2,81.0,None,"[{""check_in"": ""2024-11-02"", ""check_out"": ""2024...",25.0,£,"{"""": ""Airbnb service fee\nShow price breakdown..."
63,14132224,2024-11-03,2024-09-15 13:41:16,None,AVAILABLE,None,2,81.0,None,"[{""check_in"": ""2024-11-02"", ""check_out"": ""2024...",25.0,£,"{"""": ""Airbnb service fee\nShow price breakdown..."
64,14132224,2024-11-04,2024-09-15 13:41:16,None,AVAILABLE,None,2,81.0,None,"[{""check_in"": ""2024-11-03"", ""check_out"": ""2024...",25.0,£,"{"""": ""Airbnb service fee\nShow price breakdown..."
65,14132224,2024-11-05,2024-09-15 13:41:16,None,AVAILABLE,None,2,81.0,None,"[{""check_in"": ""2024-11-04"", ""check_out"": ""2024...",25.0,£,"{"""": ""Airbnb service fee\nShow price breakdown..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,14132224,2025-02-19,2024-09-15 13:41:16,None,AVAILABLE,None,2,72.0,None,"[{""check_in"": ""2025-02-18"", ""check_out"": ""2025...",25.0,£,"{"""": ""Airbnb service fee\nShow price breakdown..."
172,14132224,2025-02-20,2024-09-15 13:41:16,None,AVAILABLE,None,2,72.0,None,"[{""check_in"": ""2025-02-19"", ""check_out"": ""2025...",25.0,£,"{"""": ""Airbnb service fee\nShow price breakdown..."
173,14132224,2025-02-21,2024-09-15 13:41:16,None,AVAILABLE,None,2,72.0,None,"[{""check_in"": ""2025-02-20"", ""check_out"": ""2025...",25.0,£,"{"""": ""Airbnb service fee\nShow price breakdown..."
174,14132224,2025-02-22,2024-09-15 13:41:16,None,AVAILABLE,None,2,72.0,None,"[{""check_in"": ""2025-02-21"", ""check_out"": ""2025...",25.0,£,"{"""": ""Airbnb service fee\nShow price breakdown..."
